In [3]:
import os
import cv2
import numpy as np
from keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data(directory):
    images = []
    labels = []
    class_names = os.listdir(directory)
    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            images.append(img)
            labels.append(class_name)
    return np.array(images), np.array(labels)

data_directory = "E:\\win_old\\uni\\4_Diploma\\dataset_normal" #директория
X, y = load_data(data_directory)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# уменьшение размера обучающего набора данных
num_train_samples = min(500, X_train.shape[0])  #желаемое количество обучающих образцов (<~525)
np.random.seed(5)
random_indices = np.random.choice(X_train.shape[0], num_train_samples, replace=False)
print(random_indices)
X_train_subset = X_train[random_indices]
X_train_subset = X_train_subset.astype('float32') / 255.0

model = tf.keras.models.load_model("best_model.keras")

# Convert TF1 Keras model file to TF2 SavedModel.
model.export(filepath='saved_model_2/')

# Convert TF2 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_2/')
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_types = [tf.uint8]  # Использование типа данных uint8


def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(X_train_subset).batch(1).take(100):
    yield [input_value]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

with open('best_model.tflite', 'wb') as f:
  f.write(tflite_model_quant)

[138 285  53 300  52 404 191 342 132 168 281 247 199 381 153  66 317  75
 282  21 251 213 328 275  99 222  43 108 343 240 248 344 393  98 390 209
 293 416 302 155 182   9 123 141 150  50 195 327 161 402  71 372 219 321
 185  49 157  47 279 296 368 117 358 151 229  62 127 421 339 237  90 322
 295 242 107 307 272 316   0  60 269 335 122 292 224 259 115 313 101 387
  55 170 333 201 347 129 412  48 252 312  42  26 140  56  88 250 331 226
 348  12 352 354 417 212 139  76 388  89 350 286  18  24  33 283 176 261
 422 273   6  10 149 152  81 360 238 244 218 234 336 221  83  92  45 353
  46 246 198  15 280  58 407 297  59 376  39 365  25 230  84  61  51 337
 320 186  11  17  67 277 414  96  68 106   4 424 102 184 379 227 395 383
 410 415 134  93 159 374 346 241  37 119 124 163 211 111 249 166 334   2
  95 131  23   3 363 305 403  28 351 223 133 298 299 373 142 409 288 156
 263 236 396 145 291 359  85 154 174 290 116 181  79 197  74  40 367 382
 423 162 408 128 329  20 306   1 378 278 397 253 26

INFO:tensorflow:Assets written to: saved_model_2/assets


Saved artifact at 'saved_model_2/'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1080, 1080, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2436026102608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2436026103760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2436026103184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2436026104720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2436028219664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2436028220432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2436028220816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2436028221776: TensorSpec(shape=(), dtype=tf.resource, name=None)
